In [ ]:
# Colab 환경에서 나눔 폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타임 재시작이 필요합니다.
print("✅ 나눔 폰트 설치 완료. **런타임(Runtime)을 재시작(Restart Runtime)하세요.**")

In [ ]:
# 런타임 재시작 후, 아래 코드를 실행하여 Matplotlib 설정
import matplotlib.pyplot as plt

# Matplotlib의 폰트 설정을 나눔고딕으로 변경
plt.rc('font', family='NanumGothic')

# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

print("✅ Matplotlib 폰트 설정 완료. 이제 한글이 정상적으로 표시됩니다.")

# ⚠️ 이 코드 이후에 시각화 코드를 다시 실행하면 됩니다.

In [ ]:
# KoNLPy 설치
!pip install konlpy
# JAVA 환경이 필요할 수 있습니다. Colab에서는 보통 기본 설정되어 있습니다.

In [ ]:
# 필수 라이브러리 설치 (새 셀에 입력)
!pip install scikit-learn pandas numpy

## Initialize KoNLPy and Define Helper Functions

### Subtask:
KoNLPy 설치, Okt 형태소 분석기 초기화, 'clean_business_name' (상호명 정제), 'filter_franchise' (프랜차이이즈 필터링), 'analyze_morphemes' (형태소 분석), 'remove_stopwords' (불용어 제거) 함수 및 불용어 리스트('stopwords')를 통합된 코드 블록으로 정의합니다. 이는 여러 파일을 처리하기 위한 사전 준비 단계입니다.


In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from collections import Counter
import itertools
import glob
import os

# Okt 형태소 분석기 객체 생성
okt = Okt()

def clean_business_name(text):
    """
    상호명 텍스트에서 분석에 방해되는 불필요한 기호 및 수식어를 제거합니다.
    """
    if not isinstance(text, str):
        return ""

    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)

    text = re.sub(r'\d+호?점', '', text)
    text = re.sub(r'\s\w+점\b', '', text)

    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

def filter_franchise(df_input):
    """
    지역화폐 가맹점 데이터의 순수성(소상공인 중심)을 유지하기 위해
    대규모 체인점 상호를 효율적으로 필터링합니다.
    """
    franchise_keywords = ['스타벅스', 'GS25', 'CU', '세븐일레븐', '파리바게뜨', '맥도날드', '롯데마트']

    pattern = '|'.join(franchise_keywords)
    filtered_df = df_input[~df_input['상호명'].str.contains(pattern, na=False)].copy()

    return filtered_df

def analyze_morphemes(text):
    """
    텍스트를 입력받아 Okt 형태소 분석기를 사용하여 명사(Noun)만 추출합니다.
    """
    if not isinstance(text, str) or not text.strip():
        return []

    morphemes = okt.nouns(text)
    return morphemes

# 불용어(Stopwords) 정의
stopwords = {
    '점', '상호', '하우스', '집', '본점', '지점', '센터', '스토어', '마켓',
    '아파트', '빌딩', '타운', '대리점', '시티', '랜드', '월드', '파크', '가든', '프라자', '클럽',
    '마트', '백화점', '쇼핑', '몰', '아울렛', '더', '앤', '앤드', '앤', '베스트', '서비스', '전문', '제일',
    '그룹', '전', '후','지에스','씨유',
    '호', '명', '이', '가', '의', '에', '와', '과', '은', '는', '을', '를', '도', '만', '로', '와', '등',
    '가평', # 분석 대상 지역명을 불용어에 추가
    '파주', '성남' # 예시, 필요에 따라 추가
}

def remove_stopwords(morpheme_list):
    """
    형태소 리스트에서 정의된 불용어 및 한 글자 단어를 제거합니다.
    """
    filtered_list = [
        word for word in morpheme_list
        if word not in stopwords and len(word) > 1
    ]
    return filtered_list

print("KoNLPy 및 헬퍼 함수 정의 완료.")

In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from collections import Counter
import itertools
import glob
import os

# ----------------------------------------------------------------------------
# 1. Initialize KoNLPy and Define Helper Functions
# This block consolidates all necessary functions for data processing.
# ----------------------------------------------------------------------------

# Okt 형태소 분석기 객체 생성
okt = Okt()

def clean_business_name(text):
    """
    상호명 텍스트에서 분석에 방해되는 불필요한 기호 및 수식어를 제거합니다.
    """
    if not isinstance(text, str):
        return ""

    # (1) 법인/단체명 제거: (주), (유), [본점] 등 괄호와 그 안의 내용 삭제
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)

    # (2) 지점명 제거: '~점', '~호점' 등 (지역 특화 분석의 순수성을 위해 제거)
    text = re.sub(r'\d+호?점', '', text) # 1호점, 2호점 등 제거
    text = re.sub(r'\s\w+점\b', '', text) # 띄어쓰기 후 'ㅇㅇ점'으로 끝나는 단어 제거

    # (3) 특수문자 제거: 한글, 영문, 숫자, 공백을 제외한 모든 문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)

    # (4) 불필요한 공백 정리: 다중 공백을 하나의 공백으로 줄이고 양쪽 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def filter_franchise(df_input):
    """
    지역화폐 가맹점 데이터의 순수성(소상공인 중심)을 유지하기 위해
    대규모 체인점 상호를 효율적으로 필터링합니다.
    """
    # ⚠️ 여기에 분석에서 제외할 프랜차이즈 및 대기업 이름을 추가하세요.
    franchise_keywords = [
        '스타벅스', 'GS25', 'CU', '세븐일레븐', '파리바게뜨', '맥도날드', '롯데마트',
        '버거킹', 'KFC', '써브웨이', '이마트', '홈플러스', '롯데슈퍼', '노브랜드',
        '다이소', '올리브영', '랄라블라', '롭스', '배스킨라빈스', '던킨도너츠', '파스쿠찌',
        '엔제리너스', '탐앤탐스', '투썸플레이스', '빽다방', '이디야', '컴포즈커피',
        '메가커피', '맘스터치', '교촌치킨', 'BBQ', 'BHC', '굽네치킨', '네네치킨',
        '도미노피자', '피자헛', '미스터피자', '죠스떡볶이', '신전떡볶이', '엽기떡볶이',
        '국민은행', '신한은행', '우리은행', '하나은행', '농협', '기업은행', '우체국',
        '현대자동차', '기아자동차', '르노삼성', '쌍용자동차', '제네시스',
        'SK텔레콤', 'KT', 'LG유플러스', '삼성전자서비스', 'LG전자서비스',
        'LG생활건강', '아모레퍼시픽', '코웨이', '청호나이스', '교원', '웅진', '대교', '구몬',
        '쿠팡', '네이버', '카카오', '배달의민족', '요기요', '여기어때', '야놀자',
        '롯데', '신세계', '현대', 'CJ', 'GS', 'SK', '한화', '두산', '포스코', 'LG', '삼성',
        # 추가적으로 지역별 특수 프랜차이즈나 대기업 추가 가능
    ]

    # 상호명 컬럼에서 해당 키워드가 포함된 행을 제외합니다.
    pattern = '|'.join(franchise_keywords)
    # na=False: NaN 값은 필터링에서 제외되지 않도록 처리
    # 대소문자 무시를 위해 flags=re.IGNORECASE 추가
    filtered_df = df_input[~df_input['상호명'].str.contains(pattern, na=False, flags=re.IGNORECASE)].copy()

    return filtered_df

def analyze_morphemes(text):
    """
    텍스트를 입력받아 Okt 형태소 분석기를 사용하여 명사(Noun)만 추출합니다.
    """
    if not isinstance(text, str) or not text.strip():
        return []
    morphemes = okt.nouns(text)
    return morphemes

# 1. 불용어(Stopwords) 정의
# 분석 목적에 방해가 되는 일반적인 키워드나 의미 없는 단어들을 추가합니다.
# 상권 분석의 순수성을 위해 상호명에 자주 포함되지만, 특징을 나타내지 않는 단어를 포함합니다.
stopwords = {
    '점', '상호', '하우스', '집', '본점', '지점', '센터', '스토어', '마켓',
    '아파트', '빌딩', '타운', '대리점', '시티', '랜드', '월드', '파크', '가든', '프라자', '클럽',
    '마트', '백화점', '쇼핑', '몰', '아울렛', '더', '앤', '앤드', '앤', '베스트', '서비스', '전문', '제일',
    '그룹', '전', '후', '지에스', '씨유', '알파', '오피스', '넥스', '주식회사', '기업', '코리아',
    '호', '명', '이', '가', '의', '에', '와', '과', '은', '는', '을', '를', '도', '만', '로', '와', '등',
    # 분석 대상 지역명이 있다면 여기에 추가하세요.
    # 예: '파주', '성남', '가평' (이미 데이터에 존재하는 지역명도 불용어로 추가하여 일반적인 특징 파악에 유리합니다.)
    '가평' # 가평군 데이터 분석 시 '가평'이 너무 많이 나올 수 있으므로 불용어로 추가.
}

def remove_stopwords(morpheme_list):
    """
    형태소 리스트에서 정의된 불용어 및 한 글자 단어를 제거합니다.
    """
    # 불용어 제거 및 한 글자 단어 제외
    filtered_list = [
        word for word in morpheme_list
        if word not in stopwords and len(word) > 1
    ]
    return filtered_list

# ----------------------------------------------------------------------------
# 2. Identify CSV Files for Processing
# ----------------------------------------------------------------------------
# /content/ 디렉토리에서 모든 CSV 파일 목록을 찾습니다.
csv_files = glob.glob('/content/grouped_상호명_by_category/*.csv')

if not csv_files:
    print("🚨 /content/ 디렉토리에서 처리할 CSV 파일을 찾을 수 없습니다. 파일을 업로드하거나 경로를 확인해주세요.")
else:
    print(f"✅ 총 {len(csv_files)}개의 CSV 파일을 찾았습니다: {', '.join([os.path.basename(f) for f in csv_files])}\n")

    # ------------------------------------------------------------------------
    # 3. Process Each CSV File, Analyze Keywords, and Save Results
    # ------------------------------------------------------------------------
    for file_path in csv_files:
        file_name_base = os.path.basename(file_path).replace('.csv', '')
        print(f"--- '{file_name_base}.csv' 파일 처리 시작 ---")

        try:
            # 데이터 로드
            df = pd.read_csv(file_path)

            # '상호명' 컬럼이 없는 경우 건너뛰기
            if '상호명' not in df.columns:
                print(f"⚠️ '{file_name_base}.csv' 파일에 '상호명' 컬럼이 없어 건너뜁니다.")
                continue

            # 텍스트 정제 적용
            df['cleaned_name'] = df['상호명'].apply(clean_business_name)

            # 프랜차이즈 필터링 적용
            # 원본 데이터가 변경되지 않도록 .copy() 사용
            df_filtered = filter_franchise(df.copy())

            # 형태소 분석 적용
            df_filtered['morphemes'] = df_filtered['cleaned_name'].apply(analyze_morphemes)

            # 불용어 제거 적용
            df_filtered['final_keywords'] = df_filtered['morphemes'].apply(remove_stopwords)

            # 모든 최종 키워드 리스트를 하나의 리스트로 합치기
            all_final_keywords = list(itertools.chain.from_iterable(df_filtered['final_keywords']))

            # 빈도수 계산
            final_keyword_counts = Counter(all_final_keywords)

            # 가장 흔한 상위 N개 키워드 출력
            N = 30
            print(f"\n--- '{file_name_base}' 파일의 상위 {N}개 키워드 빈도 분석 결과 ---")

            if final_keyword_counts:
                print(final_keyword_counts.most_common(N))

                # 빈도 분석 결과를 데이터프레임으로 변환하여 CSV로 저장
                df_final_counts = pd.DataFrame(final_keyword_counts.most_common(),
                                               columns=['Keyword', 'Frequency'])
                output_csv_path = f'{file_name_base}_final_keyword_frequency.csv'
                df_final_counts.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
                print(f"✅ 최종 키워드 빈도 분석 결과가 '{output_csv_path}'로 저장되었습니다.")
            else:
                print("⚠️ 분석할 유효한 키워드가 없어 빈도 분석 결과가 생성되지 않았습니다.")

        except Exception as e:
            print(f"❌ '{file_name_base}.csv' 파일 처리 중 오류 발생: {e}")

        print(f"\n--- '{file_name_base}.csv' 파일 처리 완료 ---\n")

print("--- 모든 CSV 파일 처리 완료 ---")

업종별 상호명 추출 코드

In [ ]:
import pandas as pd
import os
import re

# --- 설정 변수 ---
# ⚠️ 파일 경로 설정
FILE_PATH = '지역화폐 가맹점 현황_20250801.csv'
# ⚠️ 컬럼 이름 설정 (실제 데이터에 맞게 조정 필요)
INDUSTRY_COL = '업종명(종목명)' # 그룹핑 기준 컬럼
NAME_COL = '상호명' # CSV 파일에 포함할 상호명 컬럼
# ⚠️ 출력 디렉토리 설정
OUTPUT_DIR = 'grouped_상호명_by_category'

# --- 카테고리 정의 ---
# 업종명을 기준으로 유사 그룹핑하는 카테고리 및 키워드 정의
# 업종명에 아래 키워드가 포함되면 해당 카테고리로 분류됩니다.
CATEGORIES = {
    '음식점': ['일반음식점', '휴게음식점', '분식', '뷔페', '양식', '일식', '중식', '치킨', '패스트푸드'],
    '음료/디저트': ['커피', '제과', '아이스크림', '떡', '음료'],
    '마트/편의점/유통': ['슈퍼마켓', '편의점', '마트', '유통', '식음료'],
    '병원/약국/의료': ['병원', '의원', '약국', '한의원', '치과', '의료기기', '산후조리원'],
    '미용/뷰티/위생': ['미용실', '이발소', '피부', '네일', '화장품', '목욕', '세탁'],
    '학원/교육': ['학원', '교습', '교육', '독서실'],
    '레저/스포츠': ['골프', '헬스', '스포츠', '레저', '노래방', '당구장', '수영장', '스키'],
    '숙박/여행': ['숙박', '펜션', '모텔', '호텔', '여행', '캠핑'],
    '자동차/주유': ['주유소', '충전소', '정비', '세차', '타이어', '부품', '견인'],
    '가구/인테리어/건축': ['가구', '인테리어', '철물', '건축', '조명', '페인트'],
    '의류/잡화': ['의류', '신발', '가방', '안경', '시계', '귀금속', '악세사리'],
    '농축수산물': ['정육', '축산', '수산', '농산', '청과', '비료', '사료'],
    '반려동물': ['동물병원', '애완'],
    '기타 서비스': ['부동산', '서비스', '수리', '사진', '인쇄', '광고']
}

# --- 메인 함수 ---
def group_and_export_merchants(file_path, industry_col, name_col, categories, output_dir):
    """
    CSV 파일을 읽어와 정의된 카테고리에 따라 상호명을 그룹핑하고,
    각 그룹별로 상호명만 포함된 CSV 파일을 생성합니다.
    """

    # 1. 데이터 로드 (인코딩 에러 방지)
    df = pd.DataFrame()
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"✅ 파일을 'utf-8' 인코딩으로 성공적으로 읽었습니다.")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='cp949')
            print(f"✅ 파일을 'cp949' 인코딩으로 성공적으로 읽었습니다.")
        except Exception as e:
            print(f"❌ 오류: CSV 파일을 읽을 수 없습니다. (파일 경로: {file_path})")
            print(f"자세한 오류: {e}")
            return

    if df.empty:
        print("❌ 오류: 데이터프레임이 비어 있어 작업을 진행할 수 없습니다.")
        return

    # 2. 필수 컬럼 확인
    if industry_col not in df.columns or name_col not in df.columns:
        print(f"❌ 오류: 데이터에 필수 컬럼 ('{industry_col}' 또는 '{name_col}')이 없습니다.")
        print(f"현재 컬럼 목록: {list(df.columns)}")
        return

    # 3. 그룹핑을 위한 카테고리 컬럼 생성
    print("⏳ 업종명을 기준으로 가맹점 그룹핑 중...")

    # 'Category' 컬럼을 초기화하고, 매칭되는 카테고리를 찾습니다.
    df['Category'] = '기타_미분류'

    # 업종명 문자열을 소문자/공백 제거 처리하여 매칭 정확도를 높일 수 있으나,
    # 여기서는 원본 문자열을 사용합니다.

    for category, keywords in categories.items():
        # 키워드 중 하나라도 업종명에 포함되는지 확인하는 정규 표현식 패턴 생성
        pattern = r"|".join([re.escape(k) for k in keywords])

        # 'Category' 컬럼이 아직 '기타_미분류'인 행에 대해서만 처리
        # 해당 카테고리에 매칭되는 행의 'Category' 값을 업데이트
        df.loc[
            (df['Category'] == '기타_미분류') & df[industry_col].str.contains(pattern, case=False, na=False),
            'Category'
        ] = category

    print("✅ 그룹핑 완료.")

    # 4. 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    print(f"저장 디렉토리: '{output_dir}'")

    # 5. 카테고리별로 상호명만 추출하여 CSV 파일 생성
    grouped = df.groupby('Category')
    file_count = 0

    print("⏳ 그룹별 상호명 CSV 파일 생성 중...")
    for category_name, group_data in grouped:
        # 파일 이름으로 사용 불가능한 문자를 제거하고 공백을 밑줄(_)로 대체
        safe_group_name = re.sub(r'[\\/:*?"<>|]', '', category_name).replace(' ', '_').strip('_')

        if not safe_group_name:
            safe_group_name = "미분류_그룹"

        file_name = f"{safe_group_name}.csv"
        output_path = os.path.join(output_dir, file_name)

        # 상호명 컬럼만 선택하고, 컬럼 이름을 '상호명'으로 고정
        export_df = group_data[[name_col]].rename(columns={name_col: '상호명'})

        # CSV 파일로 저장 (인덱스 제외)
        export_df.to_csv(output_path, index=False, encoding='utf-8-sig')

        file_count += 1

    print("---")
    print(f"🎉 총 {file_count}개의 카테고리별 CSV 파일이 '{output_dir}'에 생성되었습니다.")
    print("생성된 파일에는 각 그룹에 속하는 가맹점의 상호명만 포함됩니다.")

# --- 코드 실행 ---
group_and_export_merchants(FILE_PATH, INDUSTRY_COL, NAME_COL, CATEGORIES, OUTPUT_DIR)

군집분석 코드

In [ ]:
import pandas as pd
import re
import numpy as np
import json # 딕셔너리 출력을 위해 추가
from konlpy.tag import Okt
from collections import Counter
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# ----------------------------------------------------------------------------
# 1. 설정 변수
# ----------------------------------------------------------------------------
FILE_PATH = '/content/grouped_상호명_by_category/학원교육.csv' # ⚠️ 여기에 실제 데이터 파일 경로를 입력하세요.
BUSINESS_NAME_COLUMN = '상호명'# ⚠️ 상호명이 포함된 컬럼 이름 확인 후 수정
K_CLUSTERS = 5 # 분석에 사용할 군집(클러스터) 개수

# Okt 형태소 분석기 객체 생성
okt = Okt()

# ----------------------------------------------------------------------------
# 2. 전처리 함수 정의
# ----------------------------------------------------------------------------

def clean_business_name(text):
    """ 상호명 텍스트에서 불필요한 기호 및 수식어를 제거합니다. """
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    text = re.sub(r'\d+호?점', '', text)
    text = re.sub(r'\s\w+점\b', '', text)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def filter_franchise(df_input, col_name='상호명'):
    """ 대규모 체인점 상호를 필터링합니다. """
    franchise_keywords = [
        '스타벅스', '지에스', '씨유', '세븐일레븐', '파리바게뜨', '맥도날드', '롯데마트',
        '버거킹', 'KFC', '이마트', '홈플러스', '다이소', '올리브영', '배스킨라빈스',
        '던킨도너츠', '투썸플레이스', '빽다방', '이디야', '메가커피', '맘스터치', 'BBQ',
        '국민은행', '신한은행', '삼성', 'LG', 'SK', 'CJ','이마트'
    ]
    pattern = '|'.join(franchise_keywords)
    # 대소문자 무시 (flags=re.IGNORECASE)
    filtered_df = df_input[~df_input[col_name].str.contains(pattern, na=False, flags=re.IGNORECASE)].copy()
    return filtered_df

def analyze_morphemes(text):
    """ 텍스트에서 명사(Noun)만 추출합니다. """
    if not isinstance(text, str) or not text.strip():
        return []
    return okt.nouns(text)

# 불용어 정의 (분석 목적에 맞게 추가/삭제 가능)
stopwords = {
    '점', '상호', '하우스', '집', '본점', '지점', '센터', '스토어', '마켓',
    '아파트', '빌딩', '타운', '시티', '랜드', '월드', '파크', '가든', '프라자', '클럽',
    '마트', '백화점', '쇼핑', '몰', '아울렛', '더', '앤', '베스트', '서비스', '전문', '제일',
    '그룹', '전', '후', '호', '명', '이', '가', '의', '에', '와', '과', '은', '는', '을', '를',
    '도', '만', '로', '등',
    # ⚠️ 분석 대상 지역명이 있다면 여기에 추가하세요.
}

def remove_stopwords(morpheme_list):
    """ 형태소 리스트에서 정의된 불용어 및 한 글자 단어를 제거합니다. """
    filtered_list = [
        word for word in morpheme_list
        if word not in stopwords and len(word) > 1
    ]
    return filtered_list

# ----------------------------------------------------------------------------
# 3. 데이터 로드 및 전처리 파이프라인 실행
# ----------------------------------------------------------------------------

try:
    # 1) 데이터 로드
    df = pd.read_csv(FILE_PATH)

    if BUSINESS_NAME_COLUMN not in df.columns:
        print(f"🚨 지정된 컬럼명 ('{BUSINESS_NAME_COLUMN}')이 파일에 없습니다. 컬럼명을 확인하고 수정하세요.")
        exit()

    # 2) 텍스트 정제 및 프랜차이즈 필터링
    df['cleaned_name'] = df[BUSINESS_NAME_COLUMN].apply(clean_business_name)
    df_filtered = filter_franchise(df, col_name='cleaned_name')

    # 3) 형태소 분석 및 불용어 제거
    df_filtered['morphemes'] = df_filtered['cleaned_name'].apply(analyze_morphemes)
    df_filtered['final_keywords'] = df_filtered['morphemes'].apply(remove_stopwords)

    # 4) 군집분석용 문자열 생성 (keyword_string)
    df_filtered['keyword_string'] = df_filtered['final_keywords'].apply(lambda x: ' '.join(x))

    # 유효 데이터 확인
    df_analysis = df_filtered[df_filtered['keyword_string'].str.strip() != '']
    if df_analysis.empty:
        print("⚠️ 모든 전처리 후 분석할 유효한 상호명 데이터가 없습니다. 중단합니다.")
        exit()

    print(f"✅ 전처리 및 필터링 완료. 원본 데이터 {len(df)}개 중 최종 분석 데이터 {len(df_analysis)}개.")

    # ------------------------------------------------------------------------
    # 4. 군집분석 (Clustering Analysis)
    # ------------------------------------------------------------------------

    # 1) 키워드 벡터화 (TF-IDF)
    tfidf = TfidfVectorizer(min_df=5) # 최소 5개 문서에 등장하는 키워드만 사용
    tfidf_matrix = tfidf.fit_transform(df_analysis['keyword_string'])
    feature_names = tfidf.get_feature_names_out()

    print(f"\n✅ TF-IDF 행렬 생성 완료. Shape: {tfidf_matrix.shape}")

    # 2) K-Means 군집분석 적용
    actual_k = min(K_CLUSTERS, tfidf_matrix.shape[0])
    if actual_k < 2:
        print("⚠️ 군집분석을 수행하기에 유효 데이터가 너무 적습니다 (2개 미만).")
    else:
        kmeans = KMeans(n_clusters=actual_k, random_state=42, n_init=10)
        kmeans.fit(tfidf_matrix)
        df_analysis['cluster'] = kmeans.labels_

        # 원본 데이터프레임에 군집 결과 병합
        df_filtered = df_filtered.merge(df_analysis[['cluster']], left_index=True, right_index=True, how='left')
        df_filtered['cluster'] = df_filtered['cluster'].fillna(-1).astype(int) # 군집 미포함 데이터는 -1로 표시

        print(f"\n✅ K-Means 군집분석 완료 (K={actual_k})")
        print(f"군집별 데이터 개수:\n{df_analysis['cluster'].value_counts().sort_index()}")

        # 3) 군집별 특징 키워드 분석 및 시각화용 딕셔너리 출력 (수정)

        def get_top_keywords(cluster_index, top_n=10):
            """ 군집별 상위 키워드와 TF-IDF 점수를 추출합니다. """
            centroid = kmeans.cluster_centers_
            sorted_centroid_indices = centroid[cluster_index].argsort()[::-1]
            top_n_indices = sorted_centroid_indices[:top_n]
            keywords = [feature_names[i] for i in top_n_indices]
            scores = [round(centroid[cluster_index][i], 3) for i in top_n_indices]
            return list(zip(keywords, scores))

        top_keywords_by_cluster = {}
        print("\n--- 군집별 상위 특징 키워드 및 시각화용 딕셔너리 ---")

        for i in range(actual_k):
            # 상위 N개 키워드 추출 (리스트 of 튜플 형태)
            top_keywords_list = get_top_keywords(i)

            # 딕셔너리 형태로 변환: {'키워드': 점수, ...}
            keywords_dict = {keyword: score for keyword, score in top_keywords_list}
            top_keywords_by_cluster[i] = keywords_dict

            cluster_size = len(df_analysis[df_analysis['cluster'] == i])
            # 기존 출력 형식도 유지하여 중간 확인을 용이하게 합니다.
            print(f"**Cluster {i} (데이터 수: {cluster_size}개):**")
            print(top_keywords_list)

        print("\n" + "=" * 60)
        print("🚨 아래 'top_keywords_for_visualization' 변수의 내용을 복사하여 시각화 코드에 바로 붙여넣으세요.")

        # 최종 딕셔너리 변수를 선언하고 JSON 형태로 출력
        visualization_dict_output = json.dumps(top_keywords_by_cluster, ensure_ascii=False, indent=4)
        print("top_keywords_for_visualization = \\")
        print(visualization_dict_output)

        print("=" * 60)

        # 4) 최종 결과 저장
        output_data_path = 'final_clustered_business_data.csv'
        df_filtered.to_csv(output_data_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ 군집분석 결과가 포함된 최종 데이터가 '{output_data_path}'로 저장되었습니다.")

except FileNotFoundError:
    print(f"🚨 파일을 찾을 수 없습니다: '{FILE_PATH}'. 경로를 다시 확인해주세요.")
except Exception as e:
    print(f"❌ 데이터 처리 중 예상치 못한 오류 발생: {e}")

군집분석 시각화 코드

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ☠ 이 코드는 'final_clustered_business_data.csv'와 'final_keyword_frequency.csv' 파일이
#    별도로 존재하거나, 이전 단계의 군집 특징 키워드 결과를 수동으로 입력해야 합니다.
#    여기서는 임시로 군집 특징 키워드 리스트를 활용하는 예시를 보여줍니다.

# --- (예시 데이터: 실제 분석 결과로 대체해야 함) ---
# 각 클러스터별로 상위 10개 이상의 키워드가 있다고 가정하고, 시각화 시 상위 10개만 사용합니다.
top_keywords_by_cluster = {
   "0": {
        "음악학": 0.585,
        "피아노": 0.045,
        "이화": 0.024,
        "뮤직": 0.024,
        "소리": 0.021,
        "뮤즈": 0.014,
        "숙명": 0.011,
        "클랑": 0.009,
        "칸타빌레": 0.008,
        "멜로디": 0.008
    },
    "1": {
        "교습": 0.335,
        "수학": 0.263,
        "미술": 0.115,
        "음악": 0.097,
        "피아노": 0.08,
        "아트": 0.036,
        "국어": 0.017,
        "논술": 0.014,
        "그림": 0.013,
        "해법": 0.007
    },
    "2": {
        "영어": 0.491,
        "교습": 0.178,
        "뮤엠": 0.053,
        "학원": 0.051,
        "수학학원": 0.031,
        "수학": 0.025,
        "잉글리시": 0.021,
        "잉글리쉬": 0.021,
        "보습": 0.019,
        "미래엔": 0.016
    },
    "3": {
        "학원": 0.378,
        "미술": 0.075,
        "보습": 0.049,
        "수학": 0.037,
        "실용음악": 0.026,
        "아트": 0.023,
        "에듀": 0.021,
        "피아노": 0.016,
        "잉글리쉬": 0.015,
        "입시": 0.014
    },
    "4": {
        "수학학원": 0.051,
        "태권도": 0.045,
        "영어학원": 0.043,
        "스터디": 0.025,
        "카페": 0.02,
        "논술": 0.016,
        "학원": 0.015,
        "부방": 0.015,
        "교육": 0.014,
        "독서": 0.013
    }
}
# --- (예시 데이터 끝) ---

# 시각화 설정
TOP_N_KEYWORDS = 10

for cluster, keywords in top_keywords_by_cluster.items():
    # 1. 키워드를 점수 기준으로 내림차순 정렬하고 상위 N개만 선택
    sorted_keywords = sorted(keywords.items(), key=lambda item: item[1], reverse=True)[:TOP_N_KEYWORDS]

    # 2. DataFrame 생성
    keywords_df = pd.DataFrame(sorted_keywords, columns=['Keyword', 'Score'])

    # 새로운 Figure와 Axes 생성 (각 클러스터별로 별도의 그래프)
    fig, ax = plt.subplots(figsize=(6, 5)) # 적절한 크기로 조정

    # 3. 막대 그래프 그리기 (가로 막대 그래프)
    sns.barplot(
        x='Score',  # x축을 점수로 설정
        y='Keyword',  # y축을 키워드로 설정
        data=keywords_df,
        ax=ax,
        palette='viridis',
        hue='Keyword', # 'y' 변수를 hue로 설정하여 FutureWarning 해결
        legend=False # 범례 비활성화
    )

    ax.set_title(f'Cluster {cluster} 상위 {TOP_N_KEYWORDS}개 특징 키워드', fontsize=14) # 각 그래프에 제목 설정
    ax.set_xlabel('TF-IDF Score', fontsize=12)
    ax.set_ylabel('') # y축 라벨 이름 제거 (키워드 자체가 라벨이므로)

    plt.tight_layout() # 그래프 레이아웃 조정
    plt.show() # 각 그래프를 개별적으로 표시